In [1]:
from robot_descriptions.z1_description import URDF_PATH
from robot_descriptions.z1_mj_description import MJCF_PATH


import mujoco
import pinocchio as pin
import numpy as np

np.random.seed(0)

In [2]:
pinmodel = pin.buildModelFromUrdf(URDF_PATH)
pindata = pinmodel.createData()

mjmodel = mujoco.MjModel.from_xml_path(MJCF_PATH)
mjdata = mujoco.MjData(mjmodel)

In [3]:
q = np.random.randn(mjmodel.nq)
v = np.random.randn(mjmodel.nv)
dv = np.random.randn(mjmodel.nv)

# tau = pin.rnea(pinmodel, pindata, q, v, dv)

mjdata.qpos = q.copy()
mjdata.qvel = v.copy()
mjdata.qacc = dv.copy()
# mjdata.ctrl = tau.copy()

In [4]:
body_idx = 3

In [5]:
pin.computeAllTerms(pinmodel, pindata, q, v)
pin.forwardKinematics(pinmodel, pindata, q, v, dv)
# pin.framesForwardKinematics(pinmodel, pindata, q)

In [6]:
def pin_body(idx):
    vel = pin.getVelocity(pinmodel, pindata, idx, pin.LOCAL)

    accel = pin.getAcceleration(pinmodel, pindata, idx, pin.LOCAL)

    rotation = pindata.oMi[idx].rotation

    return vel, accel, rotation


vel, acc, rotation = pin_body(body_idx)

vel, acc, rotation

(  v =  0.0439583  -0.306261 -0.0295638
   w = -0.932648 -0.254576  0.181206,
   v = -0.0383893  -0.264928  0.0192288
   w = -0.700937  0.565538  0.382579,
 array([[-0.03662984, -0.9813841 , -0.18852983],
        [ 0.18717495, -0.19205531,  0.96336924],
        [-0.98164341,  0.        ,  0.19072547]]))

In [7]:
pin.getClassicalAcceleration(pinmodel, pindata, body_idx, pin.LOCAL)

  v = 0.0246332 -0.284535  0.316053
  w = -0.700937  0.565538  0.382579

In [8]:
# mujoco.mj_step(mjmodel, mjdata)
# mujoco.mj_kinematics(mjmodel, mjdata)
mujoco.mj_inverse(mjmodel, mjdata)

In [9]:
# TODO: Read more about transformations here:
# https://mujoco.readthedocs.io/en/stable/programming/simulation.html#coordinate-frames-and-transformations
# velocity computation 2 - xbody
rotlin_vel = np.zeros(6)
mujoco.mj_objectVelocity(mjmodel, mjdata, 2, body_idx + 1, rotlin_vel, 1)

# acceleration computation
rotlin_acc = np.zeros(6)
mujoco.mj_rnePostConstraint(mjmodel, mjdata)
mujoco.mj_objectAcceleration(mjmodel, mjdata, 2, body_idx + 1, rotlin_acc, 1)

# rotation matrix
rotation = mjdata.xmat[body_idx + 1].reshape(3, 3).copy()
rotlin_vel, rotlin_acc, rotation
vel_lin_muj = rotlin_vel[3:]
vel_ang_muj = rotlin_vel[:3]
acc_lin_muj_g = rotlin_acc[3:]
acc_lin_muj_class = rotlin_acc[3:] + rotation.T @ mjmodel.opt.gravity
acc_ang_muj = rotlin_acc[:3]
vel_lin_muj, vel_ang_muj, acc_lin_muj_class, acc_ang_muj, rotation

(array([ 0.04395834, -0.30626092, -0.02956378]),
 array([-0.93264804, -0.25457606,  0.18120606]),
 array([ 0.02463322, -0.28453462,  0.31605318]),
 array([-0.70093695,  0.56553825,  0.38257914]),
 array([[-3.66298407e-02, -9.81384103e-01, -1.88529833e-01],
        [ 1.87174949e-01, -1.92055313e-01,  9.63369241e-01],
        [-9.81643415e-01,  1.11022302e-16,  1.90725475e-01]]))

In [10]:
acc_muj_lin_spatial = acc_lin_muj_class - pin.skew(vel_ang_muj) @ vel_lin_muj
acc_muj_lin_spatial

array([-0.03838935, -0.26492753,  0.01922879])

In [11]:
jacobian_pin = pin.getJointJacobian(pinmodel, pindata, body_idx, pin.LOCAL)
jacobian_pin[:3], jacobian_pin[3:]

(array([[-1.73472348e-18, -2.90427808e-01,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-3.22349917e-01,  6.93889390e-18,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 6.93889390e-18,  1.95324571e-01,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]]),
 array([[-0.98164341,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  1.        ,  1.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.19072547,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ]]))

In [12]:
jacobian_muj_lin = np.zeros((3, mjmodel.nv))
jacobian_muj_rot = np.zeros((3, mjmodel.nv))

In [13]:
mujoco.mj_jacPointAxis(
    mjmodel,
    mjdata,
    jacobian_muj_lin,
    None,
    mjdata.xanchor[body_idx],
    mjdata.xaxis[body_idx],
    body_idx,
)
jac_lin = np.zeros((3, mjmodel.nv))
mujoco.mj_jacBody(mjmodel, mjdata, jac_lin, jacobian_muj_rot, body_idx + 1)
rotation = mjdata.xmat[body_idx + 1].reshape(3, 3).copy()

In [14]:
rotation.T @ jacobian_muj_lin, rotation.T @ jacobian_muj_rot

(array([[-3.67587661e-17, -2.33427808e-01,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-2.24818089e-01,  3.57384914e-17,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-6.52424616e-17, -2.26754287e-02,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]]),
 array([[-9.81643415e-01,  1.34321680e-16,  1.34321680e-16,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 1.11022302e-16,  1.00000000e+00,  1.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 1.90725475e-01,  1.62847693e-16,  1.62847693e-16,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]]))

In [15]:
rotation.T @ jac_lin

array([[-4.18136891e-17, -2.90427808e-01,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-3.22349917e-01,  1.72110774e-17,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-4.44107565e-17,  1.95324571e-01,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [16]:
(
    np.allclose(jacobian_pin[:3], rotation.T @ jacobian_muj_lin),
    np.allclose(jacobian_pin[:3], rotation.T @ jac_lin),
    np.allclose(jacobian_pin[3:], rotation.T @ jacobian_muj_rot),
)

(False, True, True)

In [ ]:
# mju_crossMotion
# subtree_com
# mju_transformSpatial